In [44]:
import PyPDF2
import openai
import nltk
import re
from nltk.tokenize import sent_tokenize

import numpy as np
import tensorflow_hub as hub
from sklearn.neighbors import NearestNeighbors

In [35]:
# Download the 'punkt' tokenizer from NLTK
nltk.download('punkt')

[nltk_data] Downloading package punkt to c:\Python310\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
with open('../api.key', 'r') as file:
    api_key = file.read().strip()

openai.api_key = api_key

In [38]:
def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)

    # Remove any extra spaces
    text = ' '.join(text.split())
    
    return text

In [39]:
path = "C:\Dropbox\HARVARD_AI\e-Book Competing in the age of AI.pdf" 
start_page = 1
num_pages = 0

sentences_list = []

reader = PyPDF2.PdfReader(path)

num_pages = len(reader.pages)
print(num_pages)

for i in range(start_page-1, num_pages):
    page = reader.pages[i]
    text = page.extract_text() 
    text = preprocess(text) 

    sentences = sent_tokenize(text)

    # Add the sentences to the list
    sentences_list.extend(sentences)


299


In [63]:
class SemanticSearch:
    
    def __init__(self):
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False
    
    def fit(self, data, batch=100, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True

    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]
        
        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors
     
    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [60]:
sentences_groups = []
for i in range(0, len(sentences_list), 5):
    group = sentences_list[i:i + 5]
    paragraph = ' '.join(group)
    sentences_groups.append(paragraph)

In [64]:
recommender = SemanticSearch()

In [65]:
recommender.fit(sentences_groups)

In [66]:
question = "What is the text about?"

In [67]:
def generate_text(prompt, engine="text-davinci-003"):
    # openai.api_key = openAI_key
    completions = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.7,
    )
    message = completions.choices[0].text
    return message

def generate_answer(question):
    topn_chunks = recommender(question)
    prompt = ""
    prompt += 'search results:\n\n'
    for c in topn_chunks:
        prompt += c + '\n\n'
        
    prompt += "Instructions: Compose a comprehensive reply to the query using the search results given. "\
              "Cite each reference using [ Page Number] notation (every result has this number at the beginning). "\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
              "with the same name, create separate answers for each. Only include information found in the results and "\
              "don't add any additional information. Make sure the answer is correct and don't output false content. "\
              "If the text does not relate to the query, simply state 'Text Not Found in PDF'. Ignore outlier "\
              "search results which has nothing to do with the question. Only answer what is asked. The "\
              "answer should be short and concise. Answer step-by-step. \n\nQuery: {question}\nAnswer: "
    
    prompt += f"Query: {question}\nAnswer:"
    answer = generate_text(prompt,"text-davinci-003")
    return answer

In [68]:
answer = generate_answer(question)

In [69]:
print(answer)

 The text is about the factors influencing value creation dynamics, network effects, AI transformation, and data platforms in digital operating models. [1] Value Creation Dynamics are impacted by the network structure of business models, which influences the underlying value or utility of a product or service. [2] Cluster analysis in the form of topic modeling is used to uncover salient topics within and across texts. [3] Microsoft's efforts to transform into an AI company are discussed, and five principles are summarized. [4] Network analysis provides a systematic way to analyze business opportunities as digital networks and AI reshape the economy. [5] Data platforms should be designed in a modular fashion to allow for decentralized innovation, and clear interfaces are key to ensuring modularity. [6]
